# Training an RMN on all session documents

In [1]:
import os
import pandas as pd

In [2]:
os.chdir("../../../scripts/assembly/")

In [3]:
from document import load_documents
from constant import DOC_ALL_PATH

In [4]:
# run several times
os.chdir("../modeling")

In [5]:
from helper import load_pickled_object
from rmn import RMN
from rmn_data_generator import RMN_DataGenerator

In [6]:
# load embedding tools
local_tools_path = '/home/rocassius/gen-data/tools'
tokenizer_dict = load_pickled_object(os.path.join(local_tools_path, "global_tokenizer_dict"))
metadata_dict = load_pickled_object(os.path.join(local_tools_path, "global_metadata_dict"))
embedding_matrix = load_pickled_object(os.path.join(local_tools_path, "global_embedding_matrix_50d"))

In [7]:
# load documents
docs_df = pd.read_csv(os.path.join(DOC_ALL_PATH, "documents_total.csv"), sep = "|")

In [8]:
# correct types
docs_df['speakerid'] = docs_df['speakerid'].astype(str)
docs_df['session'] = docs_df['session'].astype(str)

In [9]:
d = docs_df.sample(500)

In [10]:
rmn = RMN()
rmn.embedding_matrix = embedding_matrix
rmn.tokenizer_dict = tokenizer_dict
rmn.metadata_dict = metadata_dict
rmn.num_topics = 100
rmn.lamb = 1.0
rmn.build_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
data_generator = RMN_DataGenerator(rmn=rmn, data_df=docs_df, batch_size=150)

In [12]:
rmn.model.fit_generator(data_generator, epochs = 5, use_multiprocessing=True, workers=10)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
43997/43997 [==============================] - 472s 11ms/step - loss: 7.0739
Epoch 2/5
43997/43997 [==============================] - 500s 11ms/step - loss: 7.0715
Epoch 3/5
43997/43997 [==============================] - 478s 11ms/step - loss: 7.0714
Epoch 4/5
43997/43997 [==============================] - 480s 11ms/step - loss: 7.0713
Epoch 5/5
43997/43997 [==============================] - 477s 11ms/step - loss: 7.0713


In [13]:
local_model_path = "/home/rocassius/gen-data/models"
rmn.save_rmn("full", local_model_path)

In [ ]:
topics_preds = rmn.topic_model.predict_generator(data_generator, use_multiprocessing=True, workers=10)

In [ ]:
docs_df.dtypes